# Forrest imaging data slice code

In [1]:
from nipype.caching import Memory
from nipype.interfaces import fsl
from os import listdir
from os.path import isfile, join

In [15]:
# define run directory (by sub), create a file list to be merged (8 segments of movie)
run_dir = "/media/anakin/Ariel/ds000113/sub-02/ses-movie/func"
files = [join(run_dir, f) for f in listdir(run_dir) if isfile(join(run_dir, f)) and f[-11:-7] == 'bold']
files.sort()
#files # uncomment to check file list to be merged

In [16]:
# merge all segments into 1 long time course
mem = Memory(base_dir='.')
fsl_merge = mem.cache(fsl.Merge)
movie_merged = fsl_merge(dimension='t', in_files=files, tr = 2)

220313-10:52:07,745 nipype.workflow INFO:
	 [Node] Setting-up "403d2ce8588262539a6f85741e9ffa8e" in "/media/anakin/Ariel/ds000113/nipype_mem/nipype-interfaces-fsl-utils-Merge/403d2ce8588262539a6f85741e9ffa8e".
220313-10:52:07,759 nipype.workflow INFO:
	 [Node] Executing "403d2ce8588262539a6f85741e9ffa8e" <nipype.interfaces.fsl.utils.Merge>
220313-10:52:16,64 nipype.interface INFO:
	 stderr 2022-03-13T10:52:16.064176:
220313-10:52:16,71 nipype.interface INFO:
	 stderr 2022-03-13T10:52:16.064176:WARNING:: Inconsistent orientations for individual images when attempting to merge.
220313-10:52:16,73 nipype.interface INFO:
	 stderr 2022-03-13T10:52:16.064176:          Merge will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!
220313-10:52:16,74 nipype.interface INFO:
	 stderr 2022-03-13T10:52:16.064176:
220313-10:52:17,691 nipype.interface INFO:
	 stderr 2022-03-13T10:52:17.691924:
220313-10:52:17,692 nipype.interface INFO:
	 stderr 2022-03-13T10:52:17.691924:WARNIN

In [17]:
# use output file directory 
merged_filename = str(movie_merged.outputs)
merged_filename = merged_filename[15:-1]
merged_filename

'/media/anakin/Ariel/ds000113/nipype_mem/nipype-interfaces-fsl-utils-Merge/403d2ce8588262539a6f85741e9ffa8e/sub-02_ses-movie_task-movie_run-1_bold_merged.nii.gz'

In [18]:
# Splitting merged_runs to single volumes
fsl_split = mem.cache(fsl.Split)
split = fsl_split(dimension='t', in_file=merged_filename)

220313-10:53:20,243 nipype.workflow INFO:
	 [Node] Setting-up "519813f2f576f4c6bef8b8dc0308b108" in "/media/anakin/Ariel/ds000113/nipype_mem/nipype-interfaces-fsl-utils-Split/519813f2f576f4c6bef8b8dc0308b108".
220313-10:53:20,249 nipype.workflow INFO:
	 [Node] Executing "519813f2f576f4c6bef8b8dc0308b108" <nipype.interfaces.fsl.utils.Split>
220313-10:54:23,928 nipype.workflow INFO:
	 [Node] Finished "519813f2f576f4c6bef8b8dc0308b108", elapsed time 63.677291s.


In [19]:
# use output file directory and create a list of single volumes

split_name = str(split.outputs)
split_dir = split_name[15:122] #cange according to split_name

vol_list = [f for f in listdir(split_dir) if f[:3] =='vol']

# Sanity Check - length of all data
len(vol_list)

3599

In [20]:
# in order to create a 360 TR run, one must define ranges for runs
# make sure the conditions in list comprehension is accord with directories!

files = [join(split_dir, f) for f in listdir(split_dir) if isfile(join(split_dir, f)) and f[-14:-11] == 'vol']
run_merged = fsl_merge(dimension='t', in_files=files[3224:3404])

220313-10:55:19,268 nipype.workflow INFO:
	 [Node] Setting-up "3384bae20bd9a525f4103b5e54a1b4ad" in "/media/anakin/Ariel/ds000113/nipype_mem/nipype-interfaces-fsl-utils-Merge/3384bae20bd9a525f4103b5e54a1b4ad".
220313-10:55:19,399 nipype.workflow INFO:
	 [Node] Executing "3384bae20bd9a525f4103b5e54a1b4ad" <nipype.interfaces.fsl.utils.Merge>
220313-10:55:22,289 nipype.workflow INFO:
	 [Node] Finished "3384bae20bd9a525f4103b5e54a1b4ad", elapsed time 2.887917s.


In [16]:
run_merged?